In [145]:
using Flux
using Flux: onehot, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition


### Examples

In [147]:
onehot('a',['y','a','i'])

3-element Flux.OneHotVector:
 false
  true
 false

In [148]:
x= chunk([1,2,3,4,5,6],3)

3-element Array{Array{Int64,1},1}:
 [1, 2]
 [3, 4]
 [5, 6]

In [149]:
batchseq(x)

2-element Array{Array{Int64,1},1}:
 [1, 3, 5]
 [2, 4, 6]

In [150]:
x= chunk([1,2,3,4,5,6,7,8,9,10],3)

3-element Array{Array{Int64,1},1}:
 [1, 2, 3, 4]
 [5, 6, 7, 8]
 [9, 10]     

In [151]:
batchseq(x)

4-element Array{Array{T,1} where T,1}:
 [1, 5, 9]                           
 [2, 6, 10]                          
 Union{Nothing, Int64}[3, 7, nothing]
 Union{Nothing, Int64}[4, 8, nothing]

In [152]:
batchseq(x,100)

4-element Array{Array{Int64,1},1}:
 [1, 5, 9]  
 [2, 6, 10] 
 [3, 7, 100]
 [4, 8, 100]

In [222]:
u=IOBuffer()
t=wsample(['a','k','l'],[0.2,0.8,0.05])
write(u,t)
String(take!(u))

"k"

###########################################################################################

In [153]:
cd(@__DIR__)

isfile("input.txt") ||
  download("https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt",
           "input.txt")

text = collect(String(read("input.txt")));
alphabet = [unique(text)..., '_'];
text = map(ch -> onehot(ch, alphabet), text) # array of 4M and each one is 68 length
stop = onehot('_', alphabet); # array of 68 length

In [154]:
o=chunk(text, 50); # Array of 50  .. each one is 91,467 .. each one is 68 length  
@show size(o)
@show size(o[1])
@show size(o[end])
@show size(o[1][1])


size(o) = (50,)
size(o[1]) = (91467,)
size(o[end]) = (91455,)
size((o[1])[1]) = (68,)


(68,)

In [213]:
b=batchseq( chunk(text, 50),stop)
@show size(b)
@show size(b[1])
@show size(b[end])

size(b) = (91467,)
size(b[1]) = (68, 50)
size(b[end]) = (68, 50)


(68, 50)

In [156]:
d= collect(partition(b,50)); # 91467/50 -> 1830 

################################################################################################################

In [210]:
N = length(alphabet)
seqlen = 50
nbatch = 50

Xs = collect( partition( batchseq( chunk(text, nbatch), stop), seqlen));
Ys = collect(partition(batchseq(chunk(text[2:end], nbatch), stop), seqlen));



In [159]:
m = Chain(
  LSTM(N, 128),
  LSTM(128, 128),
  Dense(128, N),
  softmax)

m = gpu(m)

function loss(xs, ys)
  l = sum(crossentropy.(m.(gpu.(xs)), gpu.(ys)))
  Flux.truncate!(m)
  return l
end

opt = ADAM(params(m), 0.01)
tx, ty = (gpu.(Xs[5]), gpu.(Ys[5]))
evalcb = () -> @show loss(tx, ty)


#27 (generic function with 1 method)

In [160]:

Flux.train!(loss, zip(Xs, Ys), opt,cb = throttle(evalcb, 30))


loss(tx, ty) = 204.27115834530588 (tracked)


InterruptException: InterruptException:

In [6]:
# Sampling
m = cpu(m)

function sample(m, alphabet, len; temp = 1)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)).data)
  end
  return String(take!(buf))
end

sample(m, alphabet, 1000) |> println

# evalcb = function ()
#   @show loss(Xs[5], Ys[5])
#   println(sample(deepcopy(m), alphabet, 100))
# end


[
With you up all it.
Thy hoind to pretly not it should none
So lived's my slying hars him land.

KING CLAUDIDE:
Why my birs present, whore makest my body's Hansters reador
That you is wold rens, by the writion.

OLIVIANO:
Play you, counterabe in the Qoerity. Come, so a worbe:
To Achar, 'Twereless, sound up in my land.

CLOTEN:
Not in thind of my deed blass; and Frond, thou dreath.

DUGAN:
What my soon you adlance of think his lady. Sir.'

GYMBULLIN:
I coull from it should way for that allow,
How speaks eye to't; malied, we thee of sendened? Come,
honour 'tis sour hold good, Get the got, saying,
You deland presen: if I may ready implestane;
Solnest murder is heint me my mitches were you wilf;
The revengerous, Tomen'd, here peace,
Our mears Aceive Pouse 'gonay cut, nor great you.

FLULLIB:
The you extle; Anation to too to her before, tyrand
I'll may I be for if edd the more, poor is a pint ginciment
Chapomed in true were is will had you.
How faith the whit: the injure thy choned to you-